In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
# from surprise import SVD, SVDpp, NMF, Dataset, Reader
# from surprise.model_selection import cross_validate
import pandas as pd


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Program Files\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Program Files\Python310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Program Files\Python310\li

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Program Files\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Program Files\Python310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Program Files\Python310\li

AttributeError: _ARRAY_API not found

In [4]:
# Khởi tạo Spark session
spark = SparkSession.builder.appName("CollaborativeFiltering").getOrCreate()

# Đọc dữ liệu
hotel_comments_df = pd.read_csv("data/hotel_comments_cleaned.csv")
hotel_info_df = pd.read_csv("data/hotel_info_cleaned.csv")

# Tiền xử lý dữ liệu cho mô hình PySpark ALS
als_data = hotel_comments_df[['Hotel ID', 'Reviewer ID', 'Score']]
als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))

C:\Users\admin\AppData\Local\Temp\ipykernel_768\2474371650.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
C:\Users\admin\AppData\Local\Temp\ipykernel_768\2474371650.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))


In [6]:
# Chuyển đổi DataFrame sang Spark DataFrame
spark_df = spark.createDataFrame(als_data)

# Chia dữ liệu thành tập đào tạo và kiểm tra
(training, test) = spark_df.randomSplit([0.8, 0.2])

# Xây dựng mô hình ALS
als = ALS(maxIter=10, regParam=0.1, rank=25, userCol="Reviewer ID", itemCol="Hotel ID", ratingCol="Score", coldStartStrategy="drop", nonnegative=True)
model_als = als.fit(training)


In [7]:
reviewers = spark_df.select("Reviewer ID").distinct().count()
hotels = spark_df.select("Hotel ID").distinct().count()
numerator = spark_df.count()
display(numerator, reviewers, hotels)

25892

250

30

In [8]:
# Đánh giá mô hình
predictions = model_als.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")
predictions.show(5)

Root-mean-square error = 0.9802580836483332
+--------+-----------+-----+----------+
|Hotel ID|Reviewer ID|Score|prediction|
+--------+-----------+-----+----------+
|      19|        148|  9.2|  8.978996|
|      10|         31|  8.0|  8.842883|
|      17|         31|  8.8|  9.182246|
|      22|         31|  7.6|  9.227229|
|       1|         85| 10.0|   9.21325|
+--------+-----------+-----+----------+
only showing top 5 rows



In [9]:
# Lưu mô hình sử dụng PySpark API
model_als.save("als_model_path")

In [ ]:
# # Tải dữ liệu cho Surprise
# reader = Reader(rating_scale=(1, 10))
# data = Dataset.load_from_df(hotel_comments_df[['Reviewer ID', 'Hotel ID', 'Score']], reader)

In [ ]:
# # Huấn luyện và đánh giá mô hình SVD
# svd = SVD()
# svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# # Huấn luyện và đánh giá mô hình SVD++
# svdpp = SVDpp()
# svdpp_results = cross_validate(svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# # Huấn luyện và đánh giá mô hình NMF
# nmf = NMF()
# nmf_results = cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

- SVD có RMSE trung bình là khoảng 0.9456 và MAE trung bình là 0.7589. Thời gian fit và test khá nhanh, chỉ khoảng 0.22s và 0.02s tương ứng.
- SVD++ cho thấy RMSE trung bình là khoảng 0.9477 và MAE trung bình là 0.7662. Thời gian fit và test là 0.16s và 0.01s tương ứng.
- NMF cho kết quả RMSE trung bình là 0.9766 và MAE trung bình là 0.7942. Thời gian fit của mô hình này lâu nhất là 0.86s, nhưng thời gian test tương tự như hai mô hình còn lại.

=> Mô hình SVD là một lựa chọn phù hợp cho bài toán này, nhờ vào sự cân bằng giữa độ chính xác và thời gian huấn luyện.

In [ ]:
# # Xử dụng mô hình SVD để xây dựng hàm đề xuất

# # Chuẩn bị dữ liệu
# reader = Reader(rating_scale=(1, 10))
# data = Dataset.load_from_df(hotel_comments_df[['Reviewer ID', 'Hotel ID', 'Score']], reader)
# trainset = data.build_full_trainset()

# # Huấn luyện mô hình SVD
# svd_model = SVD()
# svd_model.fit(trainset)

# def recommend_hotels(reviewer_id, num_recommendations=5):
#     # Lấy danh sách tất cả các khách sạn
#     all_hotels = hotel_info_df['Hotel_ID'].unique()
    
#     # Lấy danh sách các khách sạn đã đánh giá bởi người dùng này
#     rated_hotels = hotel_comments_df[hotel_comments_df['Reviewer ID'] == reviewer_id]['Hotel ID'].unique()
    
#     # Lọc ra các khách sạn chưa được đánh giá bởi người dùng này
#     unrated_hotels = [hotel for hotel in all_hotels if hotel not in rated_hotels]
    
#     # Dự đoán điểm cho các khách sạn chưa được đánh giá
#     predictions = [svd_model.predict(reviewer_id, hotel).est for hotel in unrated_hotels]
    
#     # Tạo DataFrame chứa kết quả và sắp xếp theo điểm dự đoán
#     recommendations = pd.DataFrame({
#         'Hotel_ID': unrated_hotels,
#         'Predicted_Score': predictions
#     })
    
#     # Lấy top các khách sạn được đề xuất cao nhất
#     top_recommendations = recommendations.sort_values(by='Predicted_Score', ascending=False).head(num_recommendations)
    
#     # Trả về thông tin chi tiết của các khách sạn được đề xuất
#     return hotel_info_df[hotel_info_df['Hotel_ID'].isin(top_recommendations['Hotel_ID'])][['Hotel_ID', 'Hotel_Name', 'Hotel_Rank', 'Hotel_Address', 'Total_Score']]

# # Ví dụ sử dụng hàm
# print(recommend_hotels(reviewer_id='148', num_recommendations=5))

In [10]:
import pickle

# Lấy ma trận user và item factors
user_factors = model_als.userFactors.toPandas()
item_factors = model_als.itemFactors.toPandas()

# Lưu các factors này vào một file pkl
with open('als_model_factors.pkl', 'wb') as f:
    pickle.dump((user_factors, item_factors), f)

In [ ]:
# spark.stop()
